In [37]:
from openai import OpenAI
from datetime import datetime
import json
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from datetime import date
import pandas as pd
from dotenv import load_dotenv
import os
import time
from Assistant_Functions.Retreive_Data import get_spreadsheet_data
# import API_RowAppend file

load_dotenv()

SERVICE_ACCOUNT_FILE = "../SpreadsheetAPI/onlybusinessdummy-8706fb48751e.json"
SPREADSHEET_ID = os.getenv("SPREADSHEET_ID")


In [38]:
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

def get_service():

    # Authenticate and build the service
    credentials = Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=['https://www.googleapis.com/auth/spreadsheets'])
    service = build('sheets', 'v4', credentials=credentials)

    return service

class Bot:
    def __init__(self, whatsapp_number, api_key=os.getenv('OPENAI_API_KEY'), assistant_id="asst_8FWoRndfw1BUlalAHW0Xib45", thread_old=None, run_old=None, user_type="user"):
        self.client = OpenAI(api_key=api_key)
        if user_type == "user":
            self.assistant_id = assistant_id
        elif user_type == "admin":
            self.assistant_id = "asst_H8Tz1E3QJaEqA4HgRDi8ocsw"
        self.number = str(whatsapp_number)
        self.service = get_service()

        if thread_old is None:
            self.thread = self.client.beta.threads.create()
            self.insert_thread_id(self.number, self.thread.id)
        else:
            self.thread = self.client.beta.threads.retrieve(
                thread_id=thread_old)

        self.data = self.get_spreadsheet_data()

        self.assistant = self.client.beta.assistants.retrieve(
            assistant_id=self.assistant_id)
        # contains tuple of (query response) in ascending order (latest response at end of list)
        self.history = []

        print(f'Number : {self.number}')

    def insert_thread_id(self, whatsapp_num, thread_id):
        print(whatsapp_num, thread_id)
        whatsapp_num = '\'' + whatsapp_num
        values = [[whatsapp_num, thread_id]]

        RANGE_NAME = 'Sheet2'

        # Call the Sheets API to append the data
        request = self.service.spreadsheets().values().append(
            spreadsheetId=SPREADSHEET_ID,
            range=RANGE_NAME,
            valueInputOption='USER_ENTERED',
            insertDataOption='INSERT_ROWS',
            body={'values': values}
        )
        response = request.execute()

        return

    def print_history(self):
        for msg, rsp in self.history:
            print(f"User: {msg}\nBot: {rsp}\n")

    def getRowOrder(self, order_id):
        order = []
        orders_data = self.get_spreadsheet_data()

        # & (orders_data['Customer ID'] == self.number)
        order = orders_data.loc[(orders_data['Order ID'] == order_id)]

        number = '+' + list(order['Customer ID'])[0]

        if len(order) == 0:
            return 'Order not found!'
        else:
            order_val = order.to_numpy().tolist()
            return f' [order_id : {order_val[0][0]}, customer_id : {order_val[0][1]}, customer_name : {order_val[0][2]}, order date : {order_val[0][3]}, order items : {order_val[0][4]}, status : {order_val[0][5]}, Delivery address : {order_val[0][6]}, Amount : {order_val[0][7]}]'

    

    def wait_on_run(self, run, thread):

        while run.status == "queued" or run.status == "in_progress":
            run = self.client.beta.threads.runs.retrieve(
                thread_id=thread.id,
                run_id=run.id,
            )
            time.sleep(0.5)

        return run

    def insert_data_to_spreadsheet(self, values=None, order_id=None):
        # schema : [order_id, customer_id, customer_name, order date, order items, status, rider, Rider contact number, Delivery address, Amount, Rating]

        # values should be passed in the following format:
        # values = [["23", "69", "Zain Ali Khokhar", "01-03-2024", "HP Envy Screen Protector, HP Envy Hinge", "Delivered", "Sponge-Bob", "03248433434", "Out of Lahore", "$6666.44", "9", "Added through API"]]

        try:
            now = datetime.now()
            formatted_date_time = now.strftime('%d/%m/%y - %H:%M:%S')

            values[0][3] = formatted_date_time
            values[0][1] = self.number
            order_id = order_id + 1
            values[0][0] = order_id
            RANGE_NAME = 'Sheet1'

            # Authenticate and build the service
            credentials = Credentials.from_service_account_file(
                SERVICE_ACCOUNT_FILE, scopes=['https://www.googleapis.com/auth/spreadsheets'])
            service = build('sheets', 'v4', credentials=credentials)

            # Call the Sheets API to append the data
            request = service.spreadsheets().values().append(
                spreadsheetId=SPREADSHEET_ID,
                range=RANGE_NAME,
                valueInputOption='USER_ENTERED',
                insertDataOption='INSERT_ROWS',
                body={'values': values}
            )
            response = request.execute()

            return 'Successfully added order!', order_id

        except Exception as e:

            print('Failed to add order!', e)
            return "Failed to add order!", order_id

    def call_required_function(self, tools_called, order_id):

        tool_outputs = []

        print(tools_called)

        returned_id = order_id

        for tool in tools_called:
            if (tool.function.name == 'insert_data_to_spreadsheet'):
                values_param = json.loads(tool.function.arguments)['values']
                response, returned_id = self.insert_data_to_spreadsheet(
                    values=values_param, order_id=order_id)
                tool_outputs.append(
                    {'tool_call_id': tool.id, 'output': response})

            if (tool.function.name == 'get_row_order'):
                values_param = json.loads(tool.function.arguments)['order_id']
                response = self.getRowOrder(values_param)
                tool_outputs.append(
                    {'tool_call_id': tool.id, 'output': response})

        return tool_outputs, returned_id

    def send_message(self, message_content, order_id):

        message = self.client.beta.threads.messages.create(
            thread_id=self.thread.id,
            role="user",
            content=message_content,
        )
        run = self.client.beta.threads.runs.create(
            thread_id=self.thread.id,
            assistant_id=self.assistant.id,
        )
        run = self.wait_on_run(run, self.thread)

        returned_id = order_id

        print(run.status)
        if (run.status == 'requires_action'):
            tool_outputs, returned_id = self.call_required_function(
                run.required_action.submit_tool_outputs.tool_calls, order_id)

            run = self.client.beta.threads.runs.submit_tool_outputs_and_poll(
                thread_id=self.thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs
            )

        messages = self.client.beta.threads.messages.list(
            thread_id=self.thread.id)

        response = messages.to_dict()["data"][0]["content"][0]['text']['value']
        self.history.append((message_content, response))
        return response, returned_id


In [39]:

bot = Bot(whatsapp_number='+923248433434')
#bot.data.head(4)





+923248433434 thread_9NnM9bZ8kQ5d9tDAtbGNgpZe
['Order ID', 'Customer ID', 'Customer Name', 'Order Date', 'Order Items', 'Status', 'Delivery Address', 'Amount', 'Comments']
Number : +923248433434


In [29]:
response = bot.send_message("My name is Ahad and my address is 22-C,Lahore. I would like to order 144 cheeseburgers with 20 fries.", 99)
print(response)


completed
("Hello Ahad! Thank you for your order. That's a large order of 144 cheeseburgers and 20 fries. Let me confirm the details of your order and place it for you. Just a moment, please.", 99)


In [31]:
response = bot.send_message('Thats good thank you', 99)
print(response)

requires_action
[RequiredActionFunctionToolCall(id='call_kkUZ4yzdu2tfmETQxDlsyYNP', function=Function(arguments='{"values":[["123456","001","Ahad","2023-10-22","144 Cheeseburgers, 20 Fries","Pending","22-C, Lahore",""]]}', name='insert_data_to_spreadsheet'), type='function')]
('Your order has been successfully placed, Ahad! You will receive a confirmation shortly. If you need any further assistance, feel free to let me know. Enjoy your meal!', 100)


In [151]:
response = bot.send_message('I want to add 1 serving of french fries to this order')
print(response)

completed
I can assist you with that. Since your order is pending, I'll add 1 more serving of French Fries to your existing order. Your total will be updated accordingly. If you have any other requests or need further assistance, feel free to let me know.


In [164]:
response = bot.send_message('what is your menu')
print(response)

completed
Here is the menu for QuickBite Burgers:

**Burgers:**
1. Classic Beef Burger - $5.99
   Description: A juicy beef patty with lettuce, tomato, and our signature sauce.
   
2. Cheese Lover's Burger - $6.99
   Description: Beef patty with double cheddar, onions, and pickles.

**Sides:**
1. French Fries - $2.99
   Description: Crispy golden fries seasoned with sea salt.
   
2. Onion Rings - $3.49
   Description: Crunchy onion rings with a side of dipping sauce.

**Drinks:**
1. Soda - $1.99
   Description: Your choice of cola, diet cola, or lemon-lime.
   
2. Iced Tea - $2.49
   Description: Freshly brewed iced tea with a hint of lemon.

**Desserts:**
1. Chocolate Sundae - $3.99
   Description: Creamy vanilla ice cream topped with hot fudge, whipped cream, and a cherry.
   
2. Apple Pie - $4.49
   Description: Warm apple pie with a flaky crust, served with vanilla ice cream.
